In [2]:
! pip install -q kaggle
from google.colab import files

files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                  title                                          size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------  --------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sudarshan24byte/online-food-dataset                  Online Food Dataset                             3KB  2024-03-02 18:50:30          16976        334  0.9411765        
sukhmandeepsinghbrar/heart-attack-dataset            Heart Attack Dataset                           16KB  2024-03-30 05:14:35           1108         25  1.0              
divu2001/coffee-shop-sales-analysis                  Coffee Shop Sales Analysis                      2MB  2024-03-30 10:57:57           1100         27  1.0              
syedanwarafridi/vehicle-sales-data                   Vehicle Sales Data                             19MB  2024-

In [3]:
!kaggle datasets download -d gpiosenka/car-parts-40-classes

100% 300M/300M [00:17<00:00, 20.9MB/s]
100% 300M/300M [00:17<00:00, 17.5MB/s]


In [4]:
# Extracting the compressed dataset.
from zipfile import ZipFile
data_path = '/content/car-parts-40-classes.zip'
with 	ZipFile(data_path, 'r') as zip : zip.extractall()

In [5]:
import pandas as pd
import numpy as np
from numpy import random
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import matplotlib
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
sns.set_style('darkgrid')
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import albumentations as A
import time
from tqdm import tqdm
from sklearn.metrics import f1_score
from IPython.display import YouTubeVideo
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [7]:
limiter = int(input('Entrez une valeur pour le nombre maximum d\'images par classe : '))
data_path = r'car parts 50'
trainpath = os.path.join(data_path, 'train')
validpath = os.path.join(data_path, 'valid')
testpath = os.path.join(data_path, 'test')
paths = [trainpath, validpath, testpath]
names = ['train', 'valid', 'test']

for i in range(3):
    path = paths[i]
    name = names[i]
    classes = os.listdir(path)
    filepaths = []
    labels = []

    for klass in classes:
        classpath = os.path.join(path, klass)
        flist = os.listdir(classpath)

        if len(flist) > limiter:
            flist = np.random.choice(flist, limiter, replace=False)

        for f in flist:
            fpath = os.path.join(classpath, f)
            filepaths.append(fpath)
            labels.append(klass)

    Fseries = pd.Series(filepaths, name='filepaths')
    Lseries = pd.Series(labels, name='labels')

    if name == 'train':
        train_df = pd.concat([Fseries, Lseries], axis=1)
    elif name == 'valid':
        valid_df = pd.concat([Fseries, Lseries], axis=1)
    else:
        test_df = pd.concat([Fseries, Lseries], axis=1)

num_of_classes = len(classes)
print(f'test_df length= {len(test_df)}  train_df lngth= {len(train_df)}  valid_df length= {len(valid_df)}')
train_balance = train_df['labels'].value_counts()
print(f'the train_set has {num_of_classes} classes, class balance is:\n {train_balance}')

Entrez une valeur pour le nombre maximum d'images par classe : 255
test_df length= 250  train_df lngth= 8739  valid_df length= 250
the train_set has 50 classes, class balance is:
 IGNITION COIL           200
IDLER ARM               198
STARTER                 195
CARBERATOR              192
ALTERNATOR              190
RADIO                   189
OVERFLOW TANK           189
SHIFT KNOB              189
ENGINE BLOCK            188
FUEL INJECTOR           188
BRAKE PAD               188
TORQUE CONVERTER        187
OIL PRESSURE SENSOR     187
OIL PAN                 187
GAS CAP                 186
SIDE MIRROR             186
SPOILER                 185
AIR COMPRESSOR          185
CYLINDER HEAD           184
PRESSURE PLATE          182
OXYGEN SENSOR           182
THERMOSTAT              182
CLUTCH PLATE            181
INSTRUMENT CLUSTER      180
CRANKSHAFT              180
WINDOW REGULATOR        179
RIM                     179
VACUUM BRAKE BOOSTER    178
BATTERY                 177
BRAKE RO

In [8]:
''' dataset is not balanced and needs more image samples in under represented classes.
Use agmentation to balance the dataset Define a function to balance the dataset'''
def balance(df, n,column, working_dir, img_size):
    def get_augmented_image(image): # given an image this function returns an augmented image
        width=int(image.shape[1]*.8)
        height=int(image.shape[0]*.8)
        transform= A.Compose([
            A.HorizontalFlip(p=.5),
            A.Rotate(limit=30, p=.25),
            A.RandomBrightnessContrast(p=.5),
            A.RandomGamma(p=.5),
            A.RandomCrop(width=width, height=height, p=.25) ])
        return transform(image=image)['image']
    def dummy(image):
        return image

    df=df.copy()
    print('Initial length of dataframe is ', len(df))
    aug_dir=os.path.join(working_dir, 'aug')# directory to store augmented images
    if os.path.isdir(aug_dir):# start with an empty directory
        shutil.rmtree(aug_dir)
    os.mkdir(aug_dir)
    for label in df[column].unique():
        dir_path=os.path.join(aug_dir,label)
        os.mkdir(dir_path) # make class directories within aug directory
    # create and store the augmented images
    total=0
    groups=df.groupby(column) # group by class
    for label in df[column].unique():  # for every class
        msg=f'augmenting images in train set  for class {label}                                              '
        print(msg, '\r', end='')
        group=groups.get_group(label)  # a dataframe holding only rows with the specified label
        sample_count=len(group)   # determine how many samples there are in this class
        if sample_count< n: # if the class has less than target number of images
            aug_img_count=0
            delta=n - sample_count  # number of augmented images to create
            target_dir=os.path.join(aug_dir, label)  # define where to write the images
            desc=f'augmenting class {label:25s}'
            for i in range(delta):
                j= i % sample_count # need this because we may have to go through the image list several times to get the needed number
                img_path=group['filepaths'].iloc[j]
                img=cv2.imread(img_path)
                img=get_augmented_image(img)
                fname=os.path.basename(img_path)
                fname='aug' +str(i) +'-' +fname
                dest_path=os.path.join(target_dir, fname)
                cv2.imwrite(dest_path, img)
                aug_img_count +=1
            total +=aug_img_count

    print('')
    print('Total Augmented images created= ', total)
    # create aug_df and merge with train_df to create composite training set ndf
    aug_fpaths=[]
    aug_labels=[]
    classlist=sorted(os.listdir(aug_dir))
    for klass in classlist:
        classpath=os.path.join(aug_dir, klass)
        flist=sorted(os.listdir(classpath))
        for f in flist:
            fpath=os.path.join(classpath,f)
            aug_fpaths.append(fpath)
            aug_labels.append(klass)
    Fseries=pd.Series(aug_fpaths, name='filepaths')
    Lseries=pd.Series(aug_labels, name='labels')
    aug_df=pd.concat([Fseries, Lseries], axis=1)
    df=pd.concat([df,aug_df], axis=0).reset_index(drop=True)
    print('Length of augmented dataframe is now ', len(df))
    return df

In [10]:
n=200  # number of image files desired for each class
column='labels'
working_dir=r'/content/working/dir' # where to store the augment images
import os
# Vérifier si le dossier n'existe pas déjà
if not os.path.exists(working_dir):
    # Créer le dossier
    os.makedirs(working_dir)
    print(f"Le dossier '{working_dir}' a été créé avec succès.")
else:
    print(f"Le dossier '{working_dir}' existe déjà.")

img_size=(224,224)
train_df=balance(train_df, n,column, working_dir, img_size)
print(train_df)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
5000   OIL PRESSURE SENSOR  
5001   OIL PRESSURE SENSOR  
5002   OIL PRESSURE SENSOR  
5003   OIL PRESSURE SENSOR  
5004   OIL PRESSURE SENSOR  
5005   OIL PRESSURE SENSOR  
5006   OIL PRESSURE SENSOR  
5007   OIL PRESSURE SENSOR  
5008   OIL PRESSURE SENSOR  
5009   OIL PRESSURE SENSOR  
5010   OIL PRESSURE SENSOR  
5011   OIL PRESSURE SENSOR  
5012   OIL PRESSURE SENSOR  
5013   OIL PRESSURE SENSOR  
5014   OIL PRESSURE SENSOR  
5015   OIL PRESSURE SENSOR  
5016   OIL PRESSURE SENSOR  
5017   OIL PRESSURE SENSOR  
5018   OIL PRESSURE SENSOR  
5019   OIL PRESSURE SENSOR  
5020   OIL PRESSURE SENSOR  
5021   OIL PRESSURE SENSOR  
5022   OIL PRESSURE SENSOR  
5023   OIL PRESSURE SENSOR  
5024   OIL PRESSURE SENSOR  
5025   OIL PRESSURE SENSOR  
5026   OIL PRESSURE SENSOR  
5027   OIL PRESSURE SENSOR  
5028   OIL PRESSURE SENSOR  
5029   OIL PRESSURE SENSOR  
5030   OIL PRESSURE SENSOR  
5031   OIL PRESSURE SEN

In [12]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Charger le modèle VGG16 pré-entraîné
model = VGG16(weights='imagenet', include_top=False)

# Chemin du dossier contenant les images augmentées
chemin_dossier = 'working/dir/aug'

# Dictionnaire pour stocker les vecteurs de caractéristiques de toutes les classes
vecteurs_caracteristiques_par_classe = {}

# Liste des classes (nom des sous-dossiers)
classes = os.listdir(chemin_dossier)

# Parcourir chaque classe (sous-dossier)
for classe in classes:
    chemin_classe = os.path.join(chemin_dossier, classe)
    # Vérifier si c'est un dossier
    if os.path.isdir(chemin_classe):
        # Liste pour stocker les vecteurs de caractéristiques de cette classe
        vecteurs_caracteristiques_classe = []
        # Liste des fichiers dans le dossier
        fichiers = os.listdir(chemin_classe)
        # Parcourir chaque image dans le dossier
        for fichier in fichiers:
            # Construire le chemin complet de l'image
            chemin_image = os.path.join(chemin_classe, fichier)
            # Charger et prétraiter l'image
            img = image.load_img(chemin_image, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            # Passer l'image à travers le modèle VGG16
            features = model.predict(x)
            # Aplatir les activations pour obtenir un vecteur de caractéristiques
            features_vector = features.flatten()
            # Ajouter le vecteur de caractéristiques à la liste de cette classe
            vecteurs_caracteristiques_classe.append(features_vector)
        # Ajouter la liste de vecteurs de caractéristiques de cette classe au dictionnaire
        vecteurs_caracteristiques_par_classe[classe] = vecteurs_caracteristiques_classe

# Sauvegarder les vecteurs de caractéristiques dans un fichier NumPy
np.savez('vecteurs_caracteristiques.npz', **vecteurs_caracteristiques_par_classe)

1/1 [==============================] - 0s 18ms/step
